In [1]:
from orangecontrib.associate.fpgrowth import *
import Orange

ModuleNotFoundError: No module named 'orangecontrib'

# Classification rules

Videli smo, kako se dobi povezovalna pravila na redkih podatki, tokrat si bomo ogledali še postopek na polnih podatkih. 

In [2]:
data = Orange.data.Table('zoo')
data

NameError: name 'Orange' is not defined

Ker so v matriki tudi nule, bomo to upoštevali pri poimenovanju vrednosti.

In [3]:
X, mapping = OneHot.encode(data)
names = {item: ('{}={}').format(var.name, val)
                 for item, var, val in OneHot.decode(mapping, data, mapping)}
names

NameError: name 'OneHot' is not defined

Od tu naprej je postopek že poznan. Zaradi narave podatkov lahko izberemo višjo podporo in zaupanje.

In [4]:
itemsets = {}
for itemset, support in frequent_itemsets(X, 0.7):
    itemsets[itemset] = support
for rule in association_rules(itemsets, 0.8):
        left, right, support, confidence = rule
        left_str =  ', '.join(names[i] for i in sorted(left))
        right_str = ', '.join(names[i] for i in sorted(right))
        print(left_str+" -> "+right_str)

NameError: name 'frequent_itemsets' is not defined

Opravka imamo s podatki z razredom. Lahko ustvarimo pravila, ki napovedujejo razred?

V `OneHot.encode` dodamo parameter `include_class=True`, da se upošteva tudi razred.

In [5]:
X, mapping = OneHot.encode(data, include_class=True)

NameError: name 'OneHot' is not defined

Želimo postavke z >40% podpore:

In [6]:
itemsets = dict(frequent_itemsets(X, .4))
len(itemsets)

NameError: name 'frequent_itemsets' is not defined

The transaction-coded items corresponding to class values are:

In [7]:
class_items = {item 
               for item, var, _ in OneHot.decode(mapping, data, mapping) 
               if var is data.domain.class_var}
sorted(class_items)

NameError: name 'OneHot' is not defined

That makes sense as our class variable has seven values:

In [8]:
data.domain.class_var.values

NameError: name 'data' is not defined

Now we can generate all association rules that have consequent equal to one of the class values and >80% confidence (i.e. classification rules):

In [9]:
rules = [(P, Q, supp, conf) 
         for P, Q, supp, conf in association_rules(itemsets, .8) 
         if len(Q) == 1 and Q & class_items]
len(rules)
rules

NameError: name 'association_rules' is not defined

To make them more helpful, we can use mapping to transform the rules’ items back into table domain values, e.g. for first five rules:

In [10]:
names = {item: '{}={}'.format(var.name, val) 
         for item, var, val in OneHot.decode(mapping, data, mapping)}
for ante, cons, supp, conf in rules[:5]:
                              print(', '.join(names[i] for i in ante), '-->',
                                    names[next(iter(cons))],
                                    '(supp: {}, conf: {})'.format(supp, conf))

NameError: name 'OneHot' is not defined

# CN2

In [11]:
learner = Orange.classification.CN2Learner()
classifier = learner(data)

NameError: name 'Orange' is not defined

In [12]:
# consider up to 10 solution streams at one time
learner.rule_finder.search_algorithm.beam_width = 10

# continuous value space is constrained to reduce computation time
learner.rule_finder.search_strategy.bound_continuous = True

# found rules must cover at least 15 examples
learner.rule_finder.general_validator.min_covered_examples = 15

# found rules must combine at most 2 selectors (conditions)
learner.rule_finder.general_validator.max_rule_length = 2

classifier = learner(data)

NameError: name 'learner' is not defined

Induced rules can be quickly reviewed and interpreted. They are each of the form ‘if cond then predict class”. That is, a conjunction of selectors followed by the predicted class.

In [13]:
for rule in classifier.rule_list:
    print(rule, rule.curr_class_dist.tolist())

NameError: name 'classifier' is not defined

If no other rules fire, default rule (majority classification) is used. Specific to each individual rule inducer, the application of the default rule varies.